In [199]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import requests
import csv
import os

from datetime import datetime, timedelta


# Load and prepare data

In [219]:
# 读取数据

file_path = r'C:\Users\KAI\Source\kais_lstm_load_forecasting\nanjing.csv'

# 第一行做列名(header)\第一列做索引(id)\解析第二列为日期
# 参考https://www.cnblogs.com/traditional/p/12514914.html
data_raw = pd.read_csv(file_path, header=0, parse_dates=[0], infer_datetime_format=True)

# # 初始化dataframe
# df = pd.DataFrame(columns=['time'] + list(map(str, range(int(data_raw.shape[0] / 296)))))

# 生成两个列表
data = []
value = []

# 时刻表
time_list = data_raw.columns.values.tolist()[1:]


In [220]:
data_raw
time_list

['0:00',
 '0:15',
 '0:30',
 '0:45',
 '1:00',
 '1:15',
 '1:30',
 '1:45',
 '2:00',
 '2:15',
 '2:30',
 '2:45',
 '3:00',
 '3:15',
 '3:30',
 '3:45',
 '4:00',
 '4:15',
 '4:30',
 '4:45',
 '5:00',
 '5:15',
 '5:30',
 '5:45',
 '6:00',
 '6:15',
 '6:30',
 '6:45',
 '7:00',
 '7:15',
 '7:30',
 '7:45',
 '8:00',
 '8:15',
 '8:30',
 '8:45',
 '9:00',
 '9:15',
 '9:30',
 '9:45',
 '10:00',
 '10:15',
 '10:30',
 '10:45',
 '11:00',
 '11:15',
 '11:30',
 '11:45',
 '12:00',
 '12:15',
 '12:30',
 '12:45',
 '13:00',
 '13:15',
 '13:30',
 '13:45',
 '14:00',
 '14:15',
 '14:30',
 '14:45',
 '15:00',
 '15:15',
 '15:30',
 '15:45',
 '16:00',
 '16:15',
 '16:30',
 '16:45',
 '17:00',
 '17:15',
 '17:30',
 '17:45',
 '18:00',
 '18:15',
 '18:30',
 '18:45',
 '19:00',
 '19:15',
 '19:30',
 '19:45',
 '20:00',
 '20:15',
 '20:30',
 '20:45',
 '21:00',
 '21:15',
 '21:30',
 '21:45',
 '22:00',
 '22:15',
 '22:30',
 '22:45',
 '23:00',
 '23:15',
 '23:30',
 '23:45']

In [221]:
# 遍历日期列
# 逐行（逐日期）
for idx,date_ in enumerate(data_raw['date']) :
    for time in time_list:
        _date = date_.strftime(format = '%Y-%m-%d')
        _time = datetime.strptime(time,'%H:%M').strftime(format = '%H:%M')
        data.append(_date + '_' + _time)
        
# 值表
for row in range(data_raw.shape[0]):
    value.extend(data_raw.iloc[row][1:])

In [185]:
data = pd.read_csv(file_path, header=None, names=['datetime', 'load'], index_col=[0], parse_dates=[0], infer_datetime_format=True)
df = pd.DataFrame(columns=['time'] + list(map(str, range(int(data.shape[0] / 288)))))
for idx, time in enumerate(sorted(set(data.index.time))):
    df.loc[idx] = [time.strftime(format='y%H:%M:%S')] + list(data.at_time(time)['load'].values)
df.index = df['time']
df = df.drop('time', 1)
dt_df = df.diff(1, axis=1)
dt_df = dt_df.dropna(ayxis=1)
scaler = MinMaxScaler(feature_range=(-1, 1))
scaler = scaler.fit(dt_df.values.reshape(-1, 1))
dt_df = scaler.transform(dt_df)  # dt_df is now a numpy array
split_idx = int(len(dt_df) * 0.8)
train, val = dt_df[:split_idx, :], dt_df[split_idx:, :]


IndexError: list index out of range

# Training